In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import math
import quandl
import collections, functools, operator
import matplotlib.pyplot as plt  
import seaborn as seabornInstance
import statsmodels.api as sm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from operator import itemgetter
pd.set_option('display.max_columns', 500)
pd.set_option('mode.chained_assignment', None)
%matplotlib inline

In [2]:
def data_cleaning(data):
    '''
    @author: Wuding Li
    @The function cleans the data
    '''
    
    data = data[data['date'] != 19251231]
    #counts = data['date'].value_counts().to_dict()
    
    #adjust date format
    data['date'] = data['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    
    
    #remove PRC and SHROUT missing. According to the describtion, PRC missing meanings totally missing without a clue
    data['PRC'] = pd.to_numeric(data.PRC, errors='coerce')
    data = data.dropna(subset = ['PRC'])
    data['PRC'] = data['PRC'].abs()
    
    data['SHROUT'] = pd.to_numeric(data.SHROUT, errors='coerce')
    data = data.dropna(subset = ['SHROUT'])
    
    #get the medium
    NYSE = pd.read_excel('NYSE medium.xlsx')
    NYSE['date'] = pd.to_datetime(NYSE['date'])
    data['YearMonth'] = data['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE['YearMonth'] = NYSE['date'].astype(str).apply(lambda x: x[:4] + x[5:7])
    NYSE = NYSE.drop(columns=['date'])    
    data = pd.merge(data,NYSE,how = 'left',on = 'YearMonth')
    data = data[data['YearMonth'] != '192512']
    #counts = data['date'].value_counts().to_dict()
    

    data['mkt_cap'] = data['PRC']*data['SHROUT']
    data = data[data['mkt_cap']*1000>data['market cap']] #SHROUT is approximate to 10**3

    
    #Remove ADR CLosed end fund and foreign fund
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.ffill())
    data['SHRCD'] = data.groupby(['CUSIP'])['SHRCD'].transform(lambda x: x.bfill())
    data = data[data['SHRCD'].isin([11.0,10.0])]
    
    
    #check more than 5
    mask = (data['PRC'] > 5)
    data = data.loc[mask]

    #clean the RET and RETX column. 'C' stands for the first trading month. Then we change it to 0
    data['RET'] = data['RET'].apply(pd.to_numeric, errors='coerce').fillna(0)
    data['RETX'] =data['RETX'].apply(pd.to_numeric, errors='coerce').fillna(0)
    
    data = data.drop(columns=['YearMonth'])


    return data

In [3]:
def rolling_time_window(Start_Month, Start_Year, Time_Period_Training, Time_Period_Testing, Gap, data):
    '''
    @author: Wuding Li
    @The function returns two cleaned datasets. One is the sampling period data and the other is testing period data
    @variables: Start_Month: the starting month of our rolling time period
                Start_Year: the starting year of our rolling time period
                Time_Period_Training: Span of sampling Period
                Time_Period_Testing: Span of testing Period
                Gap: Gap between sampling and testing
                Data: The input data dataset
    @Also this function apply the filter no closed end fund and stock price more than 5 dollars
    
    @Updated: 10/7 fixed the end_date_test "frequency = 0" problem
    @Updated: 10/16 change "DATE" to "date" to match the updated format
    @Updated: 11/1 used the French's medium to substitute our formal medium which directly get from dataset
    '''
    
    #get the start and end date of data
    start_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=1, freq='D')[0].date())
    end_date_train = str(pd.date_range(start=str(Start_Month)+'/1/'+str(Start_Year), periods=2, freq=str(Time_Period_Training-1)+'M')[-1].date())
    mask_train = (data['date'] >= start_date_train) & (data['date'] <= end_date_train)
    
    
    start_month_testing = Start_Month+Time_Period_Training+Gap
    if start_month_testing % 12 == 0:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)-1
        start_month_testing = 12
    else:
        start_year_testing = Start_Year + math.floor(start_month_testing/12)
        start_month_testing = start_month_testing % 12    
    start_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq='D')[0].date())
    end_date_test = str(pd.date_range(start=str(start_month_testing)+'/1/'+str(start_year_testing), periods=1, freq=str(Time_Period_Testing)+'M')[-1].date())
    mask_test = (data['date'] >= start_date_test) & (data['date'] <= end_date_test)
    
    #applying filter functions below on the master dataset
    mask_master = (data['date'] >= start_date_train) & (data['date'] <= end_date_test)
    data_master = data.loc[mask_master]
    

    full = []
    for i in data_master['PERMNO'].values:
        if (data_master[data_master['PERMNO'] == i]).shape[0] == (Time_Period_Training+ Time_Period_Testing+ Gap):
            full.append(i)
    data_master = data_master[data_master['PERMNO'].isin(full)]

    
    return data_master.loc[mask_train],data_master.loc[mask_test]


In [4]:
def format_FF():
    '''
    @author: Zhikang Wang
    @The function read and clean up the fama_french dataset pulled from Dr. French's website
    @Variable: input is fama_french dataset pulled from Dr. French's website
    '''
    
    ff_data = pd.read_csv("fama_french.csv")
    # Getting and renaming the columns we need
    ff_ret = ff_data[['YearMonth','Mkt-RF','SMB','HML','RF']]
    ff_ret = ff_ret.rename(columns = {"Mkt-RF": "mkt_ret"})
    # converting the factor exposures to percentage
    ff_ret["mkt_ret"] = ff_ret["mkt_ret"]/100
    ff_ret["SMB"] = ff_ret["SMB"]/100
    ff_ret["HML"] = ff_ret["HML"]/100
    ff_ret["RF"] = ff_ret["RF"]/100
    ff_ret["YearMonth"] = ff_ret["YearMonth"].astype(str) #converting YearMonth to string
    
    return ff_ret

In [5]:
def get_weight(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)  
    @update: 11/17/2019 Wuding Li updated the weight function by Lehmann(1990)
    '''
    #count = len(set(data['PERMNO'].values))  
    data['weight_temp'] = data.groupby(['date'])['RET'].apply(lambda x: (x-x.mean()))
    data['weight_temp1']= data['weight_temp'].apply(lambda x: 0 if x<0 else x)
    data['weight_temp1'] = data['weight_temp1'].groupby(data['date']).transform('sum')
    data['weight'] = data['weight_temp']/data['weight_temp1']
    data = data.drop(columns=['weight_temp','weight_temp1'])
    data['weight'] = data.groupby(['PERMNO'])['weight'].shift(1)
    data = data.dropna(subset = ['weight'])
    return data

In [6]:
def get_weight_Residual(data):
    '''
    @author: Wuding Li
    @The function add a column to the data set that assigned the residual's weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    
    count = len(set(data['PERMNO'].values))
    #data['residual_weight']=data.groupby(['YearMonth'])['residual'].apply(lambda x: (x-x.mean())*(-1/count)).shift(1)
    #data['residual_weight']=data.groupby(['YearMonth'])['residual'].apply(lambda x: (x-x.mean())*(-1/count))
    def resd_avg(df):
        result = (df['residual'] - df['residual_mean'])
        #print(result)
        return result
    #data['residual_weight'] = data.groupby(['YearMonth']).apply(resd_avg)
    
    temp_output = pd.DataFrame()
    YearMonth_list = list(set(data['YearMonth'].values.tolist()))
    for i in YearMonth_list:
        temp = data[data['YearMonth'] == i]
        temp['residual_weight'] = (temp['residual'] - temp['residual_mean'])
        temp['residual_weight_winner'] = temp['residual_weight'].apply(lambda x: 0 if x<0 else x)
        temp['residual_real_weight'] = temp['residual_weight']/(temp['residual_weight_winner'].sum())
        temp_output = temp_output.append(temp)
        
    
    #data['residual_weight']=data.groupby(['YearMonth'])['residual_weight'].apply(lambda x: (x*-1/count))
    #data['residual_weight']=data.groupby(['YearMonth'])[['residual','residual_mean']].apply(lambda x,y: (x-y))
    temp_output = temp_output.sort_values(by=['YearMonth'])
    temp_output['residual_weight'] = temp_output.groupby(['PERMNO'])['residual_weight'].shift(1)
    temp_output = temp_output.dropna(subset = ['residual_weight'])
    #temp_output['residual_weight'] = temp_output['residual_weight']*(-1/count)
    return temp_output

In [7]:
def get_equal_weight(data):
    '''
    @author: Zhikang Wang
    @The function add a column to the data set that assigned equal weight to each stock for each month
    @Variable: data: the input dataset (Monthly data)                    
    '''
    count = len(set(data['PERMNO'].values))
    data['1'] = [1]*len(data)
    data['equal_weight']=data.groupby(['date'])['1'].apply(lambda x: x*(1/count)).shift(1)
    data = data.dropna()
    return data

In [8]:
def format_Grundy(df_conditional):
    '''
    @author: Robin Lam, Zhikang Wang
    This function formats the dataframe which is used in the conditional factor model in the spirit of Grundy.
        if excess returns on the RMRF, SMB, HML factors in month t are positive, use that factor;
        otherwise, factor = 0.
    Input:
        df_conditional: dataframe acquired after running the regression model for each stock (TOTAL)
            variables: the ones from output.csv
    Output:
        df_conditional: same dataframe as the input, but added the following: mkt_ret_UP, SMB_UP, HML_UP
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP
    '''
    df_conditional = df_conditional[['YearMonth','PERMNO','mkt_ret','SMB','HML','RF','ticker_ret','ticker_excess_ret','weight',
                                 'mkt_beta','SMB_beta','HML_beta','mkt_factor','SMB_factor','HML_factor']]

    # calculating the sum of all returns on factors and shifting all returns on factors down 1 (t-1)
    df_sum_factors = df_conditional.groupby(['YearMonth'])[['mkt_factor','SMB_factor','HML_factor']].apply(lambda x: x.sum())
    df_sum_factors = df_sum_factors.rename(columns={'mkt_factor':'mkt_factor_prev', 'SMB_factor':'SMB_factor_prev', 'HML_factor':'HML_factor_prev'})
    df_sum_factors = df_sum_factors.shift(1)
    df_combined = pd.merge(df_conditional, df_sum_factors, how='left', on=['YearMonth'])
    df_combined = df_combined.dropna()
    df_combined = df_combined.sort_values(by=['YearMonth'])

    # determining the interactive variables: RMRF_UP, SMB_UP, HML_UP
    df_combined.loc[df_combined['mkt_factor_prev'] > 0, 'mkt_ret_UP'] = df_combined['mkt_factor']    # this month, t
    df_combined.loc[df_combined['mkt_factor_prev'] <= 0, 'mkt_ret_UP'] = 0 
    df_combined.loc[df_combined['SMB_factor_prev'] > 0, 'SMB_UP'] = df_combined['SMB_factor']    # this month, t
    df_combined.loc[df_combined['SMB_factor_prev'] <= 0, 'SMB_UP'] = 0
    df_combined.loc[df_combined['HML_factor_prev'] > 0, 'HML_UP'] = df_combined['HML_factor']    # this month, t
    df_combined.loc[df_combined['HML_factor_prev'] <= 0, 'HML_UP'] = 0
    
    return df_combined

In [9]:
def regression_model(combined_temp):
    '''
    @author: Robin Lam, Yulin Chen, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        combined_temp: dataframe which contains returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        combined_temp: same dataframe as the input, but with an extra column (residual)
    
    @Update: 10/16/2019: Wuding Li Change the output date to match the paper 
    '''
    x = combined_temp[['mkt_ret','SMB','HML']]
    y = combined_temp['ticker_excess_ret']
    
    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared_adj
    
    
    # calculating the residuals for each regression model
    combined_temp['residual'] = y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] - coef[0]
    standard_dev = combined_temp['residual'].std() #get the sd of the residual
    avg_ret = combined_temp['residual'].mean()
    combined_temp['residual_sd'] = (combined_temp['residual']-avg_ret)/standard_dev
    combined_temp['residual_mean'] = combined_temp['residual_sd'].mean()
    
    combined_temp['mkt_factor'] = x['mkt_ret']*coef[1]    #used in conditiona_factor_model
    combined_temp['SMB_factor'] = x['SMB']*coef[2]    #used in conditiona_factor_model
    combined_temp['HML_factor'] = x['HML']*coef[3]    #used in conditiona_factor_model
    
    # adding coefficients to columns
    combined_temp['alpha'] = coef[0]
    combined_temp['mkt_beta'] = coef[1]
    combined_temp['SMB_beta'] = coef[2]
    combined_temp['HML_beta'] = coef[3]
    combined_temp['r_squared'] = r_squared
    
    # adding t-statistics to columns
    combined_temp['t_alpha'] = t_test[0]
    combined_temp['t_mkt_beta'] = t_test[1]
    combined_temp['t_SMB_beta'] = t_test[2]
    combined_temp['t_HML_beta'] = t_test[3]
    
    
    combined_temp['port_mkt_beta'] = combined_temp['mkt_beta']*combined_temp['weight']
    combined_temp['port_SMB_beta'] = combined_temp['SMB_beta']*combined_temp['weight']
    combined_temp['port_HML_beta'] = combined_temp['HML_beta']*combined_temp['weight']
    
      
#     keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
#     output = combined_temp[combined_temp['YearMonth'].isin(keep)]
    return combined_temp, standard_dev

In [10]:
def conditional_factor_model(temp_conditional):
    '''
    @author: Robin Lam
    This function is for running the regression in the spirit of Grundy and Martin.
    Input:
        temp_conditional: dataframe which contains returns and factor exposures
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP
    Output:
        temp_conditional: same dataframe as the input, but with an extra column (residual)
            variables: YearMonth, PERMNO, mkt_ret, SMB, HML, RF, ticker_ret, ticker_excess_ret, weight,
                       mkt_ret_UP, SMB_UP, HML_UP, residual, alpha, mkt_beta, SMB_beta, HML_beta,
                       mkt_UP_beta, SMB_UP_beta, HML_UP_beta, t_mkt_beta, t_SMB_beta, t_HML_beta,
                       t_mkt_UP_beta, t_SMB_UP_beta, t_HML_UP_beta
    '''
    x = temp_conditional[['mkt_ret','SMB','HML','mkt_ret_UP','SMB_UP','HML_UP']]
    y = temp_conditional['ticker_excess_ret']
    print(x)
    # Linear regression model
    x = sm.add_constant(x)
    regressor = sm.OLS(y, x).fit()
    # retrieving the slope
    coef = regressor.params
    # retrieving the t-statistics
    t_test = regressor.pvalues
    # retrieving the r-squared
    r_squared = regressor.rsquared_adj
    print(coef)
    
    
    # calculating the residuals for each regression model
    # temp_conditional has many rows of single PERMNO
    temp_conditional['residual'] = (y - x['mkt_ret']*coef[1] - x['SMB']*coef[2] - x['HML']*coef[3] -
                                    x['mkt_ret_UP']*coef[4] - x['SMB_UP']*coef[5] - x['HML_UP']*coef[6] - coef[0])
    standard_dev = temp_conditional['residual'].std() #get the sd of the residual
    avg_ret = temp_conditional['residual'].mean()
    temp_conditional['residual_sd'] = (temp_conditional['residual']-avg_ret)/standard_dev
    temp_conditional['residual_mean'] = temp_conditional['residual_sd'].mean()
    
    # adding coefficients to columns
    temp_conditional['alpha'] = coef[0]
    temp_conditional['mkt_beta'] = coef[1]
    temp_conditional['SMB_beta'] = coef[2]
    temp_conditional['HML_beta'] = coef[3]
    temp_conditional['mkt_UP_beta'] = coef[4]
    temp_conditional['SMB_UP_beta'] = coef[5]
    temp_conditional['HML_UP_beta'] = coef[6]
    temp_conditional['r_squared'] = r_squared
    
    # adding t-statistics to columns
    temp_conditional['t_alpha'] = t_test[0]
    temp_conditional['t_mkt_beta'] = t_test[1]
    temp_conditional['t_SMB_beta'] = t_test[2]
    temp_conditional['t_HML_beta'] = t_test[3]
    temp_conditional['t_mkt_UP_beta'] = t_test[4]
    temp_conditional['t_SMB_UP_beta'] = t_test[5]
    temp_conditional['t_HML_UP_beta'] = t_test[6]
  
    
#     # get t-2, t-1
#     keep = [sorted(list(set(temp_conditional['YearMonth'].values)))[-1],sorted(list(set(temp_conditional['YearMonth'].values)))[-2]]
#     output = temp_conditional[temp_conditional['YearMonth'].isin(keep)]
    
    return output

In [11]:
def all_regression(df):
    '''
    @author: Robin Lam, Wuding Li
    Given each stock i, this function runs the regression model.
    Input:
        df: dataframe which contains ALL of the stocks' returns and factor exposures
            variables: YearMonth, mkt_ret, SMB, HML, RF, PERMNO, ticker_ret, ticker_excess_ret
    Output:
        output: same dataframe as the input, but with an extra column (residual)
        dict_coef: dictionary of coefficients from all the regression for ALL stocks. should have 3 betas.
        
    @Update: 10/16/2019: Wuding Li Change the output format of the function and the structure of the function
    '''
    output = pd.DataFrame()
    #output_temp = pd.DataFrame()
    permno_list = list(set(df['PERMNO'].values.tolist()))
    sd_dic = {}
    for i in permno_list:
        temp_train = df[df['PERMNO'] == i]
        combined_temp, sd = regression_model(temp_train)    #there are 36 months in temp_train (individual ticker)
        #output_temp = output_temp.append(combined_temp)
        keep = [sorted(list(set(combined_temp['YearMonth'].values)))[-1],sorted(list(set(combined_temp['YearMonth'].values)))[-2]]
        output_keep = combined_temp[combined_temp['YearMonth'].isin(keep)]
        sd_dic[i] = sd
        output = output.append(output_keep)
    output = get_weight_Residual(output)   #this is also returned to merge with the test dataset for later!
    
#     '''
#     @author: Robin Lam
#     Evaluating the reversal returns using a conditional factor model (Grundy and Martin)
    
#         if excess returns on the RMRF, SMB, HML factors in month t are positive, use that factor;
#         otherwise, factor = 0.
#     '''
#     df_conditional = format_Grundy(output_temp)
#     output_conditional = pd.DataFrame()
#     permno_list_conditional = list(set(df_conditional['PERMNO'].values.tolist()))
#     #sd_dic_grundy = {}
#     for i in permno_list_conditional:
#         temp_conditional = df_conditional[df_conditional['PERMNO'] == i]    #going through each ticker
#         combined_ret = conditional_factor_model(temp_conditional)    #running Grundy regression
#         #sd_dic_grundy[i] = sd
#         output_conditional = output_conditional.append(combined_ret)
#     output_conditional = get_weight_Residual(output_conditional)    #adding residual weight

    return sd_dic, output

In [12]:
def Grundy_Table(output_conditional, monthly_Grundy):
    '''
    ------------------------- NOT USED YET! -------------------------
    @author: Robin Lam
    This function gathers and outputs the things we need to replicate Table 1.
    Input:
        output_conditional: dataframe which was generated from the Grundy regression model.
    Output:
        monthly_Grundy: a dictionary which contains information for one month.
    '''
    #monthly_Grundy = {}
    output_conditional['monthly_ret'] = output_conditional['ticker_ret']*output_conditional['weight']
    sum_ret = output_conditional['monthly_ret'].sum()
    output_conditional['monthly_residual_ret'] = output_conditional['residual']*output_conditional['residual_weight']
    sum_residual_ret = output_conditional['monthly_residual_ret'].sum()
    #monthly_Grundy['sum_ret'] = sum_ret
    #monthly_Grundy['sum_residual_ret'] = sum_residual_ret
    
    output_conditional['residual_port_alpha'] = output_conditional['alpha']*output_conditional['residual_weight']
    output_conditional['residual_port_mkt_beta'] = output_conditional['mkt_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_SMB_beta'] = output_conditional['SMB_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_HML_beta'] = output_conditional['HML_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_mkt_UP_beta'] = output_conditional['mkt_UP_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_SMB_UP_beta'] = output_conditional['SMB_UP_beta']*output_conditional['residual_weight']
    output_conditional['residual_port_HML_UP_beta'] = output_conditional['HML_UP_beta']*output_conditional['residual_weight']
    
    # Grundy table for conventional reversal strategy
    sum_port_alpha = output_conditional['port_alpha'].sum()
    sum_port_mkt_beta = output_conditional['port_mkt_beta'].sum()
    sum_port_SMB_beta = output_conditional['port_SMB_beta'].sum()
    sum_port_HML_beta = output_conditional['port_HML_beta'].sum()
    sum_port_mkt_UP_beta = output_conditional['port_mkt_UP_beta'].sum()
    sum_port_SMB_UP_beta = output_conditional['port_SMB_UP_beta'].sum()
    sum_port_HML_UP_beta = output_conditional['port_HML_UP_beta'].sum()
#     monthly_Grundy['sum_port_alpha'] = output_conditional['port_alpha'].sum()
#     monthly_Grundy['sum_port_mkt_beta'] = output_conditional['port_mkt_beta'].sum()
#     monthly_Grundy['sum_port_SMB_beta'] = output_conditional['port_SMB_beta'].sum()
#     monthly_Grundy['sum_port_HML_beta'] = output_conditional['port_HML_beta'].sum()
#     monthly_Grundy['sum_port_mkt_UP_beta'] = output_conditional['port_mkt_UP_beta'].sum()
#     monthly_Grundy['sum_port_SMB_UP_beta'] = output_conditional['port_SMB_UP_beta'].sum()
#     monthly_Grundy['sum_port_HML_UP_beta'] = output_conditional['port_HML_UP_beta'].sum()
    
    # Grundy table for residual reversal strategy
    sum_residual_port_alpha = output_conditional['residual_port_alpha'].sum()
    sum_residual_port_mkt_beta = output_conditional['residual_port_mkt_beta'].sum()
    sum_residual_port_SMB_beta = output_conditional['residual_port_SMB_beta'].sum()
    sum_residual_port_HML_beta = output_conditional['residual_port_HML_beta'].sum()
    sum_residual_port_mkt_UP_beta = output_conditional['residual_port_mkt_UP_beta'].sum()
    sum_residual_port_SMB_UP_beta = output_conditional['residual_port_SMB_UP_beta'].sum()
    sum_residual_port_HML_UP_beta = output_conditional['residual_port_HML_UP_beta'].sum()

#     monthly_Grundy['sum_residual_port_alpha'] = output_conditional['residual_port_alpha'].sum()
#     monthly_Grundy['sum_residual_port_mkt_beta'] = output_conditional['residual_port_mkt_beta'].sum()
#     monthly_Grundy['sum_residual_port_SMB_beta'] = output_conditional['residual_port_SMB_beta'].sum()
#     monthly_Grundy['sum_residual_port_HML_beta'] = output_conditional['residual_port_HML_beta'].sum()
#     monthly_Grundy['sum_residual_port_mkt_UP_beta'] = output_conditional['residual_port_mkt_UP_beta'].sum()
#     monthly_Grundy['sum_residual_port_SMB_UP_beta'] = output_conditional['residual_port_SMB_UP_beta'].sum()
#     monthly_Grundy['sum_residual_port_HML_UP_beta'] = output_conditional['residual_port_HML_UP_beta'].sum()
    
    monthly_Grundy = monthly_Grundy.append({'sum_ret': sum_ret, 'sum_residual_ret': sum_residual_ret,
                                           'sum_port_alpha': sum_port_alpha, 'sum_port_mkt_beta': sum_port_mkt_beta,
                                           'sum_port_SMB_beta': sum_port_SMB_beta, 'sum_port_HML_beta': sum_port_HML_beta,
                                           'sum_port_mkt_UP_beta': sum_port_mkt_UP_beta,
                                           'sum_port_SMB_UP_beta': sum_port_SMB_UP_beta,
                                           'sum_port_HML_UP_beta': sum_port_HML_UP_beta,
                                           'sum_residual_port_alpha': sum_residual_port_alpha,
                                           'sum_residual_port_mkt_beta': sum_residual_port_mkt_beta,
                                           'sum_residual_port_SMB_beta': sum_residual_port_SMB_beta,
                                           'sum_residual_port_HML_beta': sum_residual_port_HML_beta,
                                           'sum_residual_port_mkt_UP_beta': sum_residual_port_mkt_UP_beta,
                                           'sum_residual_port_SMB_UP_beta': sum_residual_port_SMB_UP_beta,
                                           'sum_residual_port_HML_UP_beta': sum_residual_port_HML_UP_beta},
                                           ignore_index=True)
                                            
    return monthly_Grundy

In [13]:
msf = pd.read_csv('msf.csv')
msf_before = msf

In [14]:
msf = data_cleaning(msf)

In [112]:
'''
Author: Zhikang Wang / Yulin Chen
MAIN FUNCTION.
Loop over the period from 1986 to generate rolling regression
'''

test_port = pd.DataFrame() #container for test df
output_conditional_output = pd.DataFrame()    #container for Grundy Table output
total_train_output = pd.DataFrame()

dates = []
ff = []
start = 1926      #1926

for i in range(890,1086): #（7，348） （348，659） （659，890）  (890，1086）
    if (i%12) != 0:
        month = i%12
        year = i // 12 + start
    if (i%12) == 0: 
        month = 12
        year = i // 12 + start -1

    ff_ret = format_FF()
    train,test = rolling_time_window(month, year, 37, 1, 0, msf)
    #train = get_equal_weight(train)
    train = get_weight(train)
    
    train = train.rename(columns = {"RET": "ticker_ret", "date": "YearMonth"})
    train = train[['YearMonth','PERMNO','ticker_ret','weight']]
    train["PERMNO"] = train["PERMNO"].astype(str)
    test["PERMNO"] = test["PERMNO"].astype(str)
    train["YearMonth"] = train["YearMonth"].astype(str)
    train['YearMonth'] = train['YearMonth'].apply(lambda x: x[:4] + x[5:7])
    # Merging the two dataset into one combined return dataset, which will be used as regression input
    combined_ret = pd.merge(ff_ret, train, how='right', on=['YearMonth'])
    # calculates the excess return of tickers by subtracting the returns by risk free rate
    combined_ret['ticker_excess_ret'] = combined_ret['ticker_ret'] - combined_ret['RF']
    #append dates
    month_list=["01","02","03","04","05","06","07","08","09","10","11","12"]
    month=month_list[month-1]
#     year = year + 3
    str_date = str(year)+str(month)
    dates.append(str_date)
        
    
    sd_dic, output = all_regression(combined_ret)
    
    output_conditional_output = output_conditional_output.append(output)
    total_train_output = total_train_output.append(output)
#     monthly_Grundy = pd.DataFrame()
    monthly_Grundy = pd.DataFrame(columns=['sum_ret','sum_residual_ret','sum_port_alpha','sum_port_mkt_beta',
                                          'sum_port_SMB_beta','sum_port_HML_beta','sum_port_mkt_UP_beta',
                                          'sum_port_SMB_UP_beta','sum_port_HML_UP_beta','sum_residual_port_alpha',
                                          'sum_residual_port_mkt_beta','sum_residual_port_SMB_beta',
                                          'sum_residual_port_HML_beta','sum_residual_port_mkt_UP_beta',
                                          'sum_residual_port_SMB_UP_beta','sum_residual_port_HML_UP_beta'])
    #monthly_Grundy = Grundy_Table(output_conditional, monthly_Grundy)
    #output_conditional_output = pd.concat([output_conditional_output, monthly_Grundy])
    
#     sd_df = pd.DataFrame.from_dict(sd_dic, orient='index')
#     sd_df = sd_df.reset_index()
#     sd_df =  sd_df.rename(columns = {"index": "PERMNO", 0: 'std'})
#     test = pd.merge(test,sd_df, how = 'left', on  =['PERMNO'])
    
    # Testing portfolio
#     count = len(set(output['PERMNO'].values))
#     output['test_conv_weight']=output['ticker_ret'].apply(lambda x: (x-output['ticker_ret'].mean())*(-1/count))
#     output['test_red_weight'] = output['residual'].apply(lambda x: (x-output['residual'].mean())*(-1/count))
#     output = output[['PERMNO','test_conv_weight','test_red_weight']]
#     test = pd.merge(test,output,how = 'left',on=['PERMNO'])
    test_port = test_port.append(test)
    print(i)

890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085


In [77]:
'''
    @author: Robin Lam
    Evaluating the reversal returns using a conditional factor model (Grundy and Martin)
    
        if excess returns on the RMRF, SMB, HML factors in month t are positive, use that factor;
        otherwise, factor = 0.
'''
# df_conditional = output_conditional_output
# df_conditional = df_conditional[['YearMonth','PERMNO','mkt_ret','SMB','HML','RF',
#                                     'ticker_ret','ticker_excess_ret','weight',
#                                     'mkt_factor','SMB_factor','HML_factor']]

# # calculating the sum of all returns on factors and shifting all returns on factors down 1 (t-1)
# df_sum_factors = df_conditional.groupby(['YearMonth'])[['mkt_factor','SMB_factor','HML_factor']].apply(lambda x: x.sum())
# df_sum_factors = df_sum_factors.rename(columns={'mkt_factor':'mkt_factor_prev', 'SMB_factor':'SMB_factor_prev', 'HML_factor':'HML_factor_prev'})
# df_sum_factors = df_sum_factors.shift(1)
# df_combined = pd.merge(df_conditional, df_sum_factors, how='left', on=['YearMonth'])
# df_combined = df_combined.dropna()
# df_combined = df_combined.sort_values(by=['YearMonth'])


# df_combined.loc[df_combined['mkt_factor_prev'] > 0, 'mkt_ret_UP'] = df_combined['mkt_ret']    # this month, t
# df_combined.loc[df_combined['mkt_factor_prev'] <= 0, 'mkt_ret_UP'] = 0 
# df_combined.loc[df_combined['SMB_factor_prev'] > 0, 'SMB_UP'] = df_combined['SMB']    # this month, t
# df_combined.loc[df_combined['SMB_factor_prev'] <= 0, 'SMB_UP'] = 0
# df_combined.loc[df_combined['HML_factor_prev'] > 0, 'HML_UP'] = df_combined['HML']    # this month, t
# df_combined.loc[df_combined['HML_factor_prev'] <= 0, 'HML_UP'] = 0
# df_combined = df_combined.drop(columns=['mkt_factor','SMB_factor','HML_factor'])

# print(df_combined[df_combined['YearMonth']=='193303'])

df_conditional = format_Grundy(output_conditional_output)

output_conditional = pd.DataFrame()
YearMonth_list = list(set(df_conditional['YearMonth'].values.tolist()))
for i in YearMonth_list:
    print(i)
    temp = df_conditional[df_conditional['YearMonth'] == i]    #going through each ticker
    combined_ret = conditional_factor_model(temp)    #running Grundy regression
    output_conditional = output_conditional.append(combined_ret)
#output_conditional = get_weight_Residual(output_conditional)    #adding residual weight
output_conditional

193302
     mkt_ret     SMB     HML  mkt_ret_UP    SMB_UP    HML_UP
158  -0.1524 -0.0275 -0.0273   -0.131085 -0.008257 -0.001140
258  -0.1524 -0.0275 -0.0273   -0.119602  0.016611 -0.005281
259  -0.1524 -0.0275 -0.0273   -0.137712  0.017645 -0.040525
260  -0.1524 -0.0275 -0.0273   -0.144079 -0.020660 -0.026498
261  -0.1524 -0.0275 -0.0273   -0.151924 -0.000405  0.003706
262  -0.1524 -0.0275 -0.0273   -0.145805 -0.002697  0.000315
263  -0.1524 -0.0275 -0.0273   -0.205748 -0.009533  0.007050
264  -0.1524 -0.0275 -0.0273   -0.130329 -0.033141  0.004406
265  -0.1524 -0.0275 -0.0273   -0.122686 -0.005763 -0.038174
266  -0.1524 -0.0275 -0.0273   -0.105146 -0.028754  0.005389
267  -0.1524 -0.0275 -0.0273   -0.172292  0.002569  0.013768
268  -0.1524 -0.0275 -0.0273   -0.133349  0.007918  0.003649
269  -0.1524 -0.0275 -0.0273   -0.092071  0.002300  0.005329
270  -0.1524 -0.0275 -0.0273   -0.232475  0.013379  0.017768
271  -0.1524 -0.0275 -0.0273   -0.113639 -0.003756  0.007589
272  -0.1524 -0.0

C:\Users\Robin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


IndexError: index out of bounds

In [113]:
export_csv1 = total_train_output.to_csv ('train_output4.csv', index = None, header=True)
export_csv2 = test_port.to_csv ('test_output4.csv', index = None, header=True)
export_csv3 = output_conditional_output.to_csv ('conditional_output4.csv', index = None, header=True)